In [1]:
import os
import sys
from tqdm.notebook import tqdm
import cv2
import mtcnn

sys.path.append('../../')

from video_proc import Person
from video_proc.utils import put_text

## Set up input & output files

In [2]:
input_video = '../../video4.mp4'
output_video = 'video4_out5.mp4'
face_min_height_scale = 17


## Function for frame processing

In [3]:
face_detector = mtcnn.MTCNN()

def extend_bbox(bbox, padding=0.6):
    (x, y, w, h) = bbox

    w = max(w, h)
    h = max(w, h)

    x = max(0, x - int(w * padding))
    y = max(0, y - int(h * padding))

    w = min(frame_width - x, w + int(w * padding * 2))
    h = min(frame_height - y, h + int(h * padding * 2))

    return x, y, w, h

def face_detect_mtcnn(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    detections = face_detector.detect_faces(img_rgb)

    faces = []
    if len(detections) > 0:
        for d in detections:
            x, y, w, h = d["box"]

            if (h > face_min_height) and d['confidence'] > 0.96:
                if w < 80 and h < 80:
                    x, y, w, h = extend_bbox((x, y, w, h))
                faces.append((x, y, w, h))

    return faces

## First run on video for detecting all faces

In [4]:
cap = cv2.VideoCapture(input_video)

fps = cap.get(cv2.CAP_PROP_FPS)
fps = round(fps)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

face_min_height = int(frame_height // face_min_height_scale)

fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

In [5]:
count = 0
frame_face_count = 0
persons = []
pbar = tqdm(total=500)

while (cap.isOpened()):
    cap.set(1, count)
    ret, img = cap.read()

    if ret and (count < 500):
        faces = face_detect_mtcnn(img)
        
        for x, y, w, h in faces:
            face = img[y:y + h, x:x + w]
            new_person = True
            for person in persons:
                if person.verify_person(face):
                    new_person = False
                    person.add_picture(face)
                    img = put_text(img, (x, y, w, h), person, 500)
            if new_person:
                persons.append(Person(face))
                img = put_text(img, (x, y, w, h), persons[-1], 500)
        
        
        persons = persons[-5:]
        
        out.write(img)

        count += 1
        pbar.update()

    else:
        break

cap.release()
pbar.close()
out.release()

error: OpenCV(4.5.1) /private/var/folders/nz/vv4_9tw56nv9k3tkvyszvwg80000gn/T/pip-req-build-39p1qqfs/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [6]:
out.release()